In [39]:
import pandas as pd
import os
# from sentence_transformers import SentenceTransformer
# from sklearn.cluster import AgglomerativeClustering
from tqdm import tqdm

In [42]:
df=pd.read_stata("D://stata//BWdisclosure audits//2024 June cleaning analysis 7 countries//240619.dta")
df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())
CPCleanedUnique=list(df.CPCleaned.unique())

C:\Users\Chunyun Li\AppData\Local\Temp\ipykernel_19012\3448851165.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())


In [40]:
def assign(q,orig:bool=False, cp="delete"):
    global df
    if orig==True:
        df.loc[df.Q==q,"CPCleaned"]=cp
    else:
        df.loc[df.QCleaned==q,"CPCleaned"]=cp
        
def getOriginalQ(cleanedQ):
    global df
    return df[df.QCleaned==cleanedQ].Q.value_counts()

def undo(combinedQ:str):
    global df
    df.loc[df.QCleaned==combinedQ,"QCleaned"]=df.loc[df.QCleaned==combinedQ,"Q"]

def groupSent(sentences, threshold=0.75):
    # Load pre-trained Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')
    embeddings = model.encode(sentences,show_progress_bar=False)
    clustering = AgglomerativeClustering(n_clusters=None, affinity='cosine', linkage='average', distance_threshold=1-threshold)
    clustering.fit(embeddings)
    clusters = {}
    for sentence_id, cluster_id in enumerate(clustering.labels_):
        if cluster_id not in clusters:
            clusters[cluster_id] = []
        clusters[cluster_id].append(sentences[sentence_id])
    return [group for group in clusters.values()]

genSubRule=lambda lol: {x: sorted(t, key=lambda x: len(x))[0] for t in lol for x in t}

def getQNum(qstr):
    global df
    return len(df[df.QCleaned==qstr])

def tabCountry(q):
    global df
    return df[df.QCleaned==q].Country.value_counts()

# with open("grpdQ.txt","r") as txt:
#     grpdQ=eval(txt.read())

def replace(qset, brief=None):
    '''list of questions, replace all with shortest/given'''
    qset=[x.strip() for x in qset]
    global df
    if brief==None:
        brief=sorted(qset,key=lambda x : len(x))[0]
    rplcidx=list(df[df["QCleaned"].isin(qset)].index)
    df.loc[rplcidx,"QCleaned"]=brief
    # df.loc[rplcidx,"cleaned"]=1

def getCountry(q):
    global df
    return df[df.QCleaned==q].Country.unique()

def CPVC(n):
    print(CPCleanedUnique[n])
    return df[df.CPCleaned==CPCleanedUnique[n]].QCleaned.value_counts()

def discard(q):
    global df
    df.drop(df[df.QCleaned==q].index, inplace=True)

def getYear(q):
    return df[df.QCleaned==q].AssesmentDate.str[-4:].unique()

def collapse(qcleaned, country):
    global df
    return df[(df.QCleaned==qcleaned) & (df.Country==country)].Q.value_counts()

def tab(cpnum=None,cp=None):
    assert cpnum!=None or cp!=None, "pass in at least 1 param."
    if cpnum!=None:
        cp=CPCleanedUnique[cpnum]
    print(cp)
    global df
    filtered_df=df[df['CPCleaned']==cp]
    return pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose()

def discardYear(qcleaned, country, year:int=2021):
    global df
    df.loc[(df.QCleaned==qcleaned) & (df.Country==country) & (df.AssesmentDate.str[-4:]==str(year)),"CPCleaned"]="delete"

In [149]:
df.head(3)

,index,Country,FactoryAssessedID,AssesmentDate,QuestID,CP,Q,Finding,Tags,disclosed,CPCleaned,QCleaned,id,year
0,611407,Cambodia,1029,06/06/2016,15496,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,611407,2016
1,589245,Vietnam,2151,15/12/2020,40869,overtime wages,does the employer pay workers correctly for al...,0,,0.0,overtime wages,correct pay for overtime at weekly rest days,589245,2020
2,610455,Cambodia,807,18/05/2021,42608,overtime wages,does the employer pay workers double their nor...,0,Internal: CMB Public Disclosure,0.0,overtime wages,correct pay for overtime at weekly rest days,610455,2021


In [37]:
tab(cp='osh management systems')


osh management systems


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Assess OSH issues,584,2503,161,851,474,122,1802,6497
does the employer record work-related accidents?,584,2499,161,851,474,119,1802,6490
does the factory have a written osh policy?,584,2412,0,833,474,119,1802,6224
has legal construction permits/structural safety certificate,584,2354,161,851,474,119,1802,6345
mechanisms/committee for cooperation on osh,584,2503,161,851,474,122,1802,6497
All,2920,12271,644,4237,2370,601,9010,32053


In [6]:
replace(["has the employer done an initial industrial hygiene risk assessment, a risk map, and annual risk assessments?","has the employer performed an assessment of general occupational safety and health issues in the factory?"],"Assess OSH issues")
#combined Nicaragua item with others

In [8]:
replace(["has legal construction permits/structural safety certificate","does the employer have osh technical organizational regulations (tor) that are approved by the ministry of labor?"])
#combined one in Nicaragua with others

In [12]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Jordan"),"QCleaned"]="has legal construction permits/structural safety certificate"
##move one in Jordan to others

In [29]:
replace(["does the employer develop an osh plan annually?","does the employer have an osh license?"],"does the factory have a written osh policy?")
#move one item in Vietnam to others

In [17]:
replace(["does the factory employ the required osh specialists?","has the employer appointed an officer trained in firefighting, rescue and first aid?"])

In [23]:
discard("does the employer have an steam generator license?") #one unique in Nicaragua
discard("does the employer report both accidents and non-occurrences to the ministry of labor?") #one unique item in Nicaragua

In [27]:
replace(["has the employer set up a properly functioning osh collaborators network?"],"does the factory employ the required osh specialists?")
#move one in Vietnam to similar others on osh personnel

In [31]:
df.loc[(df.QCleaned=="does the factory have a valid business license?") & (df.Country=="Bangladesh"),"QCleaned"]="does the factory have a written osh policy?"
##move one in Bangladesh to others

In [36]:
for i in ["does the employer have the required certificates for the installation/ operation of electrical installations and other machines and equipment?",
       "does the employer inspect and measure the environmental conditions in the workplace on a annual basis?",
       "does the employer investigate work-related accidents and indicate the technical recommendations necessary to prevent them?",
       "does the employer regularly inspect and maintain machines, equipment, buildings and stores?",
       "has the employer designated a person exclusively devoted to osh promotion, prevention and protection?",
       "has the employer elaborated an osh training program?",
       "does the factory employ the required osh specialists?"]:
    assign(i)


In [ ]:
assign("does the employer comply with requirements on hiv/aids?")

In [12]:
getOriginalQ("mechanisms/committee for cooperation on osh")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?                                           2558
has the employer set up a properly functioning unit in charge of osh and/or osh council?                                                             1252
does the factory have a functioning safety committee?                                                                                                 584
has the employer set up a properly functioning unit in charge of osh and/or labour protection council?                                                550
does the factory have an osh committee ?                                                                                                              498
has the employer formed a joint worker/management osh committee?                                                                                      474
does the factory have an osh committee?                                   

In [13]:
collapse("mechanisms/committee for cooperation on osh","Cambodia")

Q
has the employer developed mechanisms to ensure cooperation between workers and management on osh matters?    2503
Name: count, dtype: int64

In [ ]:
# df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the workplace have a functioning hiv/aids committee?"

In [14]:
# export
df.to_csv("D://audits620.csv", index=False)

---

In [47]:
tab(cp='wage information, use and deduction')

wage information, use and deduction


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
does the employer keep only one accurate payroll record?,584,2503,161,851,474,122,1802,6497
has the employer made any unauthorized deductions from wages?,584,2503,161,833,474,119,1802,6476
inform workers about wage pay and deduction,584,2503,161,0,474,122,1802,5646
All,1752,7509,483,1684,1422,363,5406,18619


In [46]:
replace(["does the employer keep only one accurate payroll record/ book?","does the employer keep only one accurate payroll record?"])

In [58]:
tab(cp="documentation and protection of young workers")

documentation and protection of young workers


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Keep a record of workers under age of 18,584,2237,161,833,474,119,1802,6210
Verify the age of workers before hiring,584,2503,161,851,0,122,1802,6023
All,1168,4740,322,1684,474,241,3604,12233


---

In [50]:
discard("does the employer remediate child labour as required under the cba?")
#99 deleted; one unique item asked in some years in Jordan

In [52]:
replace(["does the employer have a verification system in place to ensure they do not hire child labour in accordance with the national law?",
        "does the employer reliably verify the age of workers prior to hiring?",
         "does the employer cross-check worker's age and documents as stipulated by national law?"],"Verify the age of workers before hiring")

In [54]:
replace(["does the employer keep a record of workers under 18 years of age?",
         "does the employer comply with documentation requirements for workers under age of 18?",
         "does the employer have permission from the parents or legal guardians of workers between the ages of 14 and 16, and approval from the ministry of labor?"],
         "Keep a record of workers under age of 18")

In [57]:
discard("does the employer comply with requirements regarding the certificate of fitness for workers under age 18?")
#1 unique item in Bangladesh
discard("does the employer require workers under age 18 to provide a medical certificate, and an employment certificate or permit delivered by the director of labor?")
# 1 unique item in Haiti

In [74]:
tab(cp="overtime")

overtime


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with legal limits on overtime hours,584,2503,161,851,0,122,1802,6023
Get permission/consent for overtime,584,2259,80,833,0,122,0,3878
is overtime voluntary?,584,2503,161,833,474,119,1802,6476
All,1752,7265,402,2517,474,363,3604,16377


In [60]:
replace(["is overtime work limited to two hours per day?",
         "is overtime limited to three hours per day and nine hours per week?",
         "is overtime limited to 3 hours per day, 14 hours per week?",
         "is overtime in line with legal limits?",
         "does the employer comply with daily limits on overtime hours worked?	",
         "does the employer comply with limits on overtime hours worked?",
         "is overtime worked only for reasons allowed by law?"],"Comply with legal limits on overtime hours")

In [62]:
replace(["does the employer get permission from the labour inspector before workers work overtime?",
         "does the employer get written consent from women to work at night?","Get permission/consent for overtime"])

In [ ]:
replace(["does the employer comply with yearly limits on overtime hours worked?","Get permission/consent for overtime"])
# combine one unique item in Vietnam with dissimilar others....

In [64]:
discard("does the employer provide compensatory days off when workers work on weekly rest days?") ##1 unique item in Bangladesh
discard("does the employer schedule overtime on public holidays only under conditions permitted by law?") # 2 strange entries in Nicaragua
discard("does the employer obtain authorization from the department of labour before working on sundays?") #only asked in 2015-2018 in Haiti
discard("does the employer get permission from the labor inspector before suspending weekly rest days?") # 1 item only asked in Camboia

In [66]:
df.loc[(df.QCleaned=="does the employer prepare written instructions on overtime?") & (df.Country=="Indonesia"),"QCleaned"]="Get permission/consent for overtime"
# move 1 unique item in Indonesia to others


In [68]:
for i in ["does the employer comply with monthly limits on overtime hours worked?",
           "does the employer comply with the monthly limits on overtime hours worked?"]:
    discard(i)
#one unique question in Vietnam

In [73]:
discard("does the employer prepare written instructions on overtime?") 

In [99]:
tab(cp="welfare facilities")

welfare facilities


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Adequate changing rooms/locker,584,2383,161,833,474,117,1802,6354
Adequate eating area,584,2368,161,833,474,122,1802,6344
adequate washing facilities,584,2503,161,851,474,122,1802,6497
does the workplace have adequate accessible toilets?,584,2279,161,833,474,122,1802,6255
providing drinking water,584,2503,161,851,474,122,1802,6497
All,2920,12036,805,4201,2370,605,9010,31947


In [76]:
replace(["does the workplace have adequate hand washing taps?","adequate washing facilities	"])

In [93]:
replace(["does the workplace provide all workers a place to store their clothes?",
         "does the workplace have adequate changing rooms/lockers to meet the needs of workers?",
         "does the workplace have adequate shelters or rest rooms?",
         "does the workplace have all required facilities?"],"Adequate changing rooms/locker")

In [84]:
replace(["does the workplace have an adequate lunch room, and/or canteen?",
         "does the workplace have an adequate eating area?",
         "does the employer comply with requirements regarding canteen?",
         "does employer provide cups or other sanitary means\nfor drinking water?"],
         "Adequate eating area")

In [86]:
replace(["does the workplace have a functioning and accessible nursing room?","Adequate changing rooms/locker"])
#combine 1 unique item in Cambodia with others

In [90]:
discard("does the employer keep food samples for 24 hours?")

In [96]:
df.loc[(df.QCleaned=="is the workplace clean and tidy?"),"CPCleaned"]="working environment"

In [98]:
discard("when provided as in-kind payment, does the employer give workers enough food of decent quality?") # 1 unique in Joradn
discard("does the workplace have adequate day care facilities?") #only asked in Bangladesh, Cambodia, and Jordan


In [117]:
tab(cp="health services and first aid")

health services and first aid


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Provide first aid training for workers,584,2286,161,0,474,119,1802,5426
Workers have medical checks as required by law,584,2386,161,834,474,122,1802,6363
does the employer address safety and health risks to pregnant or nursing workers?,584,2236,161,833,0,122,1802,5738
does the employer comply with requirements on hiv/aids?,584,0,0,833,0,119,1802,3338
does the employer provide required medical services?,584,2503,161,833,474,0,1802,6357
has the employer ensured there are a sufficient number of readily accessible first aid boxes/supplies in the workplace?,584,2271,161,833,474,122,1802,6247
All,3504,11682,805,4166,1896,604,10812,33469


In [101]:
replace(["has the employer trained workers on first aid and formed a first-aid team?",
         "has the employer provided rescue and first-aid training for workers?",
         "has the employer provided first-aid training for workers?"],
         "Provide first aid training for workers")

In [111]:
replace(["do workers receive free periodic health checks as required by law?",
         "do workers have annual medical checks?",
         "do workers undergo a medical examination before being employed?"],
         "Workers have medical checks as required by law")

In [106]:
df.loc[df.Q.str.contains("annual medical checks"), "QCleaned"]="Workers have annual medical checks Haiti"

In [ ]:
df.loc[df.Q=="does the workplace have a functioning hiv/aids committee?", "QCleaned"]="does the employer comply with requirements on hiv/aids?"

In [113]:
getOriginalQ("does the employer comply with requirements on hiv/aids?")

Q
does the employer comply with the law on hiv/aids prevention and control?    1802
does the employer comply with requirements on hiv/aids?                       851
does the employer comply with national policy on hiv/aids?                    566
does the employer comply with requirements on hiv/aids ?                      119
Name: count, dtype: int64

In [114]:
replace(["does the workplace have an adequately equipped and staffed infirmary?"],
        "does the employer provide required medical services?")
##move 1 in Cambodia to others

In [116]:
for i in ["does the workplace have adequate trained first aid officers?",
"does the employer provide adequate first-aid facilities?",
"do workers have a medical check within the first three months of hiring ?",
"Workers have annual medical checks Haiti"]:
    assign(i)

#unique items only in Haiti or Indonesia

In [118]:
tab(cp="union operations")

union operations


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
employer deducts union dues as requested,584,2240,161,833,474,119,1802,6213
employer provides facility for union,584,2503,0,833,474,0,1802,6196
union has access to workers,584,2503,161,851,474,122,1802,6497
All,1752,7246,322,2517,1422,241,5406,18906


In [128]:
tab(cp="regular hours")

regular hours


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Attendance records reflect hours actuallyed worked,584,2260,129,835,474,122,1802,6206
Comply with rest periods,584,2243,161,0,474,238,1802,5502
Regular daily work hours exceed legal limit,584,2503,161,851,474,120,1802,6495
All,1752,7006,451,1686,1422,480,5406,18203


In [120]:
replace(["do regular daily and weekly working hours exceed legal limits (40 hours per week; 7 hours a day, 6 days a week or 8 hours per day, 5 days a week)?",
         "do regular daily or weekly working hours exceed the legal limit (8 hours per day, 48 hours per week)?",
         "do regular daily or weekly working hours exceed the legal limit?",
         "do regular daily working hours exceed legal limits?",
         "do regular working hours exceed 10 hours per day, or 48 hours per week?",
         "do regular daily working hours exceed 10 hours?",
         "do regular working hours exceed 11 hours per day or 48 hours per week?"],
         "Regular daily work hours exceed legal limit")

In [122]:
replace(["do the attendance records reflect the hours actually worked?",
         "do the working time records reflect the hours actually worked?"],
         "Attendance records reflect hours actuallyed worked")

In [124]:
replace(["does the employer comply with daily break periods?",
         "does the employer provide required weekly rest periods?"],
         "Comply with rest periods")

In [127]:
for i in ["does the employer provide required breastfeeding breaks?",
          "does the employer post the factory's working hours (for all shifts, and including break times)?",
          "does the employer comply with shift work requirements?",
          "do workers have at least one day off per week?"]:
    assign(i)

In [126]:
discard("do regular work shifts exceed the daily legal limit?")

In [148]:
tab(cp="contracting procedures")

contracting procedures


Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
Comply with limits on probationary period,584,2241,161,833,474,0,1802,6095
Comply with requirements on subcontracted/outsourced workers,584,1091,0,833,474,119,1802,4903
Limits on the use of fixed contracts or have contracts,584,2503,0,833,474,0,1802,6196
do workers pay any recruitment fees?,0,2236,0,833,474,0,0,3543
does the employer comply with requirements for temporary transfers of workers to new work?,0,0,0,0,0,0,1802,1802
All,1752,8071,161,3332,1896,119,7208,22539


In [130]:
replace(["do workers have to pay anyone to get a job?","do workers pay any recruitment fees?"])

In [132]:
replace(["does the employer comply with limits on the trial period for apprentices?",
         "is the probationary period applied only to permanent workers (pkwtt) and for no longer than 3 months?	",
         "is the probationary period limited to 3 months?",
         "Comply with limits on probationary period"])

In [136]:
replace(["does the employer comply with requirements concerning sub-contracted workers at the workplace?",
         "have you found non-compliance with legal requirements for compensation, contracts, osh and/or working time pertaining to non-production workers and/or sub-contracted workers?",
         "does the employer comply with legal requirements relating to outsourced workers?",
         "does the employer comply with national law concerning dispatched workers at the workplace?"],
         "Comply with requirements on subcontracted/outsourced workers")

In [140]:
replace(["limits on the use of fixed term contracts",
         "does the employer consider workers on limited duration contracts who work beyond their term as unlimited duration contract workers?",
         "do all workers (other than apprentices, badli or casual workers) have a service book?	"],
         "Limits on the use of fixed contracts or have contracts")

In [142]:
replace(["do the migrant workers have valid work permits and residence ids?"],
        "Comply with requirements on subcontracted/outsourced workers")
#move 1 in Jordan to others: on requirement for special type of workers

In [144]:
for i in ["has the employer taken sufficient steps to ensure that migrant workers do not pay any unauthorized fees?",
          "does the employer pay for the return trip of migrant workers who have been expelled from the country because of an invalid residence id?",
          "does the employer include the entire period of continuous employment when determining workers' entitlements to maternity leave, attendance bonus, seniority bonus, and/or annual leave?",
          "does the employer comply with the legal requirement relating to compensation for the non-permanent workers?",
          "does the employer comply with requirements for apprentices?",
          "does the employer comply with requirements concerning home workers?",
          "does the employer comply with legal requirements when subcontracting part of its work to another enterprise?"]:
    assign(i)